In [2]:
# --------------------------------------------------------------------
# -- IMPORT PACKAGES
# --------------------------------------------------------------------

#from openslide import open_slide
import openslide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from openslide.deepzoom import DeepZoomGenerator
import os
import imageio
from PIL import Image
import datetime
import json
from shapely.geometry import Polygon
from descartes import PolygonPatch
import cv2
import tensorflow


main_directory = "C:/Users/Administrateur/ProjetM1" 


# ----------------------------------------------------------------------------
# -- MAIN LOOP : runs code for each WSI in the folder
# ----------------------------------------------------------------------------

# select WSI folder path
WSIfolder_path = main_directory + '/WSIs'

# create WSI files list 
WSI_list = [WSI for WSI in os.listdir(WSIfolder_path) if os.path.isfile(os.path.join(WSIfolder_path, WSI))]
print("liste des WSI :", WSI_list) 

for WSI in WSI_list:

    t0 = datetime.datetime.now()
    print("Start time:", t0)
    
    name_WSI, ext_WSI = os.path.splitext(WSI)
    
    found = False
    cancerous = False
    # cancerous annotations folder
    annotation_folder_path = main_directory + '/annotations/cancerous'
    annot_list = [annot for annot in os.listdir(annotation_folder_path) if os.path.isfile(os.path.join(annotation_folder_path, annot))]
    #print("liste des annotations cancer :", annot_list) 
    for annot in annot_list:
        name_annot, ext_annot = os.path.splitext(annot)
        if name_WSI == name_annot and ext_WSI != ext_annot:
            annotation_file = annot
            found = True
            cancerous = True
    
    if found == False:
        # try healthy annotations folder
        annotation_folder_path = main_directory + '/annotations/healthy'
        annot_list = [annot for annot in os.listdir(annotation_folder_path) if os.path.isfile(os.path.join(annotation_folder_path, annot))]
        #print("liste des annotations healthy :", annot_list) 
        for annot in annot_list:
            name_annot, ext_annot = os.path.splitext(annot)
            if name_WSI == name_annot and ext_WSI != ext_annot:
                annotation_file = annot
                found = True
                
    if found == False:
        annotation_file = 'no corresponding annotation file for ' + WSI
        print("JE PASSE AU PROCHAIN WSI") 
        continue
        
           
    print("selected image :", WSI)
    print("selected annotation :", annotation_file)
    print("True = cancer, False = healthy or no annotation -->", cancerous)
    
    #load WSI
    slide = openslide.open_slide(main_directory + '/WSIs/' + WSI)
    
    # --------------------------------------------------------------------
    # 1--  CREATION TISSUE MASK
    # --------------------------------------------------------------------

    # Dimensions de l'image
    width, height = slide.dimensions
    print("The dimensions of the " + WSI + " slide at level 0 are :\n Width : " + str(width) + "\n Height : " + str(height))

    # Niveaux de zoom disponibles
    levels = slide.level_count

    # Niveau de zoom souhaité
    level = 4 #3

    # Charger l'image à ce niveau
    image = slide.read_region((0, 0), level, slide.level_dimensions[level])

    # Convertir l'image en un tableau numpy
    image = np.array(image)

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Seuillage
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Opération de morphologie : fermeture
    kernel = np.ones((3, 3), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Opération de morphologie : ouverture
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations=2)

    # Enregistrer le masque en tant qu'image
    name_WSI, ext_WSI = os.path.splitext(WSI)
    cv2.imwrite(main_directory + '/masks/tissue/' + name_WSI + '_tissue_mask.tif', opening)
    

    # --------------------------------------------------------------------
    # 2-- CREATION ANNOTATED AREA MASK
    # --------------------------------------------------------------------

    slide_dims =  slide.level_dimensions[4]

    # Find path of annotations file
    name_annot, ext_annot = os.path.splitext(annotation_file)
    if cancerous == True:
        folder_path = '/annotations/cancerous/'
    else:
        folder_path = '/annotations/healthy/'

    # load file
    try:
        with open(main_directory + folder_path + name_annot + '.geojson') as f:
            jsonData = json.load(f)
    except:
        print('Failed!')
        print('ERROR while opening the JSON file!!')
        print(all_masks_dict) 

    # Only proceed if jsonData is not empty
    if len(jsonData) != 0:
        l_jsonData = list(jsonData)
        slide_key = l_jsonData[0]  # We assume there is only one initial key
        annotation_key = l_jsonData[1]
        pointsValues = []
        for i in range(len(jsonData[slide_key])):
            for zone in jsonData[annotation_key]:
                xi = [((points[0])) for points in zone]
                yi = [((points[1])) for points in zone]
                zone_pts = np.column_stack((xi, yi))
                pointsValues.append(zone_pts)

        # Make sure that we can draw valid contours. Certain masks (not endo or epi) have only one point.
        if len(pointsValues) > 0:
            # Resize the points
            scale = 1/16
            resized_pts = []
            for c_pts in pointsValues:
                resized_pts.append(np.squeeze(c_pts))
            resized_pts = [(scale * _pts).astype(int) for _pts in resized_pts]

            mask = np.zeros((slide_dims[1], slide_dims[0]), dtype=np.uint8)
            for c in range(len(pointsValues)):
                cv2.drawContours(mask, resized_pts, c, (255, 255, 255), -1)  # '-1' is a flag to filled the contour

            # Save
            cv2.imwrite(main_directory + '/masks/annotated_area/' + name_WSI + '_annotarea_mask.tif', mask)    
    
    
    # --------------------------------------------------------------------
    # 3-- LOAD NECESSARY FILES WITH OPENSLIDE
    # --------------------------------------------------------------------    
    
    # load tissue mask
    mask_tissue_path = main_directory + '/masks/tissue/' + name_WSI + '_tissue_mask.tif'
    mask_tissue = openslide.open_slide(mask_tissue_path)
    
    # load area mask
    mask_annotarea_path = main_directory + '/masks/annotated_area/' + name_WSI + '_annotarea_mask.tif'
    mask_annotarea = openslide.open_slide(mask_annotarea_path)
    
    
    
    # ----------------------------------------------------------------------------
    # 4-- PATCH EXTRACTION : tissue mask + cancer area mask + save
    # ----------------------------------------------------------------------------
    
    # main directory to save patches
    savingdir = main_directory + "/data_saved"

    #size of patches
    patch_size = (512, 512)

    # other parameters
    threshold = 204
    level0_dim = slide.level_dimensions[0]
    #test = 0

    # Read patch by patch according to coodinates (with WSI level 0 dimensions)
    for y in range(0, level0_dim[1], patch_size[1]):
        for x in range(0, level0_dim[0], patch_size[0]):
            #remove borders
            if x == 0 or x == 512 or x ==1024 or x==level0_dim[0] or x == level0_dim[0]-patch_size[0] or x == level0_dim[0]-2*patch_size[0] or y == 0 or y == 512 or y ==1024 or y==level0_dim[1] or y == level0_dim[1]-patch_size[1] or y == level0_dim[1]-2*patch_size[1] :
                continue
            
            ###________________TISSUE MASK___________________

            # masking the image with tissue mask : go through mask (acccording to patch by patch coordinates)
            mask_patch = mask_tissue.read_region((int(x/16),int(y/16)), 0, (int(patch_size[0]/16),int(patch_size[1]/16)))
            mask_arr = np.array(mask_patch)

            # calculate color average of the patch
            average = mask_arr.mean()
            print(WSI +" Average pixel value for TISUE mask patch (%d,%d) is: %.2f" % (x, y, average))

            # if patch is a white area in mask_tissue ( = tissue)
            if average > threshold:

                ###________________CANCER AREA INSIDE MASK (cancerous annotations)___________________

                if cancerous == True: #selected annotations folder = cancerous then inside of annotated area are cancerous
                #inside white area = cancer
                
                    # masking the image with area mask 
                    mask_patch = mask_annotarea.read_region((int(x/16),int(y/16)), 0, (int(patch_size[0]/16),int(patch_size[1]/16)))
                    mask_arr = np.array(mask_patch)

                    # calculate color average of the patch
                    average = mask_arr.mean()
                    print(WSI +" Average pixel value for ANNOTATED AREA mask patch (%d,%d) is: %.2f" % (x, y, average))

                    # if patch is in a white area of mask_annotarea ( = cancerous)
                    if average > threshold:

                        # read corresponding region in WSI
                        patch = slide.read_region((x, y), 1, patch_size)

                        # saving WSI patch in CANCEROUS TISSUE directory
                        tile_dir = savingdir + "/cancerous_patches"
                        tile_name = os.path.join(tile_dir, '%d_%d' % (x, y))
                        print(WSI +" Now saving tile CANCEROUS TISSUE DIRECTORY with title: ", tile_name)
                        patch.save(tile_name + ".png")


                    # else not in cancerous area
                    else: 

                        # read corresponding region in WSI
                        patch = slide.read_region((x, y), 1, patch_size)                      

                        # saving patch in HEALTHY TISSUE directory and name
                        tile_dir = savingdir + "/healthy_patches"
                        tile_name = os.path.join(tile_dir, '%d_%d' % (x, y))
                        print(WSI +" Now saving tile in HEALTHY TISSUE DIRECTORY with title: ", tile_name)
                        patch.save(tile_name + ".png")



                ###________________CANCER AREA OUTSIDE MASK (healthy inside annotations)___________________     

                if cancerous == False : #selected annotations folder = healthy then inside of annotated area are healthy 
                #inside white area = healthy
                
                    # masking the image with area mask 
                    mask_patch = mask_annotarea.read_region((int(x/16),int(y/16)), 0, (int(patch_size[0]/16),int(patch_size[1]/16)))
                    mask_arr = np.array(mask_patch)

                    # calculate color average of the patch
                    average = mask_arr.mean()
                    print(WSI +" Average pixel value for ANNOTATED AREA mask patch (%d,%d) is: %.2f" % (x, y, average))

                    # if patch is in a white area of mask_annotarea ( = healthy)
                    if average > threshold:

                        # read corresponding region in WSI
                        patch = slide.read_region((x, y), 1, patch_size)

                        # saving WSI patch in HEALTHY TISSUE directory
                        tile_dir = savingdir + "/healthy_patches"
                        tile_name = os.path.join(tile_dir, '%d_%d' % (x, y))
                        print(WSI + " Now saving tile HEALTHY TISSUE DIRECTORY with title : ", tile_name)
                        patch.save(tile_name + ".png")


                    # else not in healthy area
                    else: 

                        # read corresponding region in WSI
                        patch = slide.read_region((x, y), 1, patch_size)                      

                        # saving patch in CANCEROUS TISSUE directory and name
                        tile_dir = savingdir + "/cancerous_patches"
                        tile_name = os.path.join(tile_dir, '%d_%d' % (x, y))
                        print(WSI + " Now saving tile in CANCEROUS TISSUE DIRECTORY with title : ", tile_name)
                        patch.save(tile_name + ".png")
                
                    
    # closing 
    mask_tissue.close()
    mask_annotarea.close()
    slide.close()

    # verification prints
    t1 = datetime.datetime.now()
    print("End time:", t1)
    print("Elapsed time for " + WSI + " is : ", t1-t0)
    print("\n\n")

liste des WSI : ['001A.tif', '011A.tif', '061D.tif', '100K.tif']
Start time: 2023-03-29 15:37:32.468240
JE PASSE AU PROCHAIN WSI
Start time: 2023-03-29 15:37:32.468240
selected image : 011A.tif
selected annotation : 011A.geojson
True = cancer, False = healthy or no annotation --> True
The dimensions of the 011A.tif slide at level 0 are :
 Width : 66832
 Height : 161152
['slide', 'annotations']
8
32
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
10072
32
011A.tif Average pixel value for TISUE mask patch (1536,1536) is: 63.75
011A.tif Average pixel value for TISUE mask patch (2048,1536) is: 63.75
011A.tif Average pixel value for TISUE mask patch (2560,1536) is: 63.75
011A.tif Average pixel value for TISUE mask patch (3072,1536) is: 63.75
011A.tif Average pixel value for TISUE mask patch (3584,1536) is: 63.75
011A.tif Average pixel value for TISUE mask patch (4096,1536) is: 63.75
011A.tif Average pixel value for TISU

011A.tif Average pixel value for TISUE mask patch (34816,2560) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (34816,2560) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\34816_2560
011A.tif Average pixel value for TISUE mask patch (35328,2560) is: 243.42
011A.tif Average pixel value for ANNOTATED AREA mask patch (35328,2560) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\35328_2560
011A.tif Average pixel value for TISUE mask patch (35840,2560) is: 183.47
011A.tif Average pixel value for TISUE mask patch (36352,2560) is: 216.71
011A.tif Average pixel value for ANNOTATED AREA mask patch (36352,2560) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\36352_2560
011A.tif Average pixel value for TISUE ma

011A.tif Average pixel value for TISUE mask patch (34816,3072) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (34816,3072) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\34816_3072
011A.tif Average pixel value for TISUE mask patch (35328,3072) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (35328,3072) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\35328_3072
011A.tif Average pixel value for TISUE mask patch (35840,3072) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (35840,3072) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\35840_3072
011A.tif Average pixel value for TISUE mask patch (36352,3072) is: 255.00
011A.tif Average pixel value for ANNOTATE

011A.tif Average pixel value for TISUE mask patch (34816,3584) is: 206.07
011A.tif Average pixel value for ANNOTATED AREA mask patch (34816,3584) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\34816_3584
011A.tif Average pixel value for TISUE mask patch (35328,3584) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (35328,3584) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\35328_3584
011A.tif Average pixel value for TISUE mask patch (35840,3584) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (35840,3584) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\35840_3584
011A.tif Average pixel value for TISUE mask patch (36352,3584) is: 255.00
011A.tif Average pixel value for ANNOTATE

011A.tif Average pixel value for TISUE mask patch (33280,4096) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (33280,4096) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\33280_4096
011A.tif Average pixel value for TISUE mask patch (33792,4096) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (33792,4096) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\33792_4096
011A.tif Average pixel value for TISUE mask patch (34304,4096) is: 245.66
011A.tif Average pixel value for ANNOTATED AREA mask patch (34304,4096) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\34304_4096
011A.tif Average pixel value for TISUE mask patch (34816,4096) is: 238.56
011A.tif Average pixel value for ANNOTATE

...

011A.tif Average pixel value for TISUE mask patch (53248,134144) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (53248,134144) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\53248_134144
011A.tif Average pixel value for TISUE mask patch (53760,134144) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (53760,134144) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\53760_134144
011A.tif Average pixel value for TISUE mask patch (54272,134144) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (54272,134144) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\54272_134144
011A.tif Average pixel value for TISUE mask patch (54784,134144) is: 255.00
011A.tif Average pixe

011A.tif Average pixel value for TISUE mask patch (52224,134656) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (52224,134656) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\52224_134656
011A.tif Average pixel value for TISUE mask patch (52736,134656) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (52736,134656) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\52736_134656
011A.tif Average pixel value for TISUE mask patch (53248,134656) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (53248,134656) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\53248_134656
011A.tif Average pixel value for TISUE mask patch (53760,134656) is: 255.00
011A.tif Average pixe

011A.tif Average pixel value for TISUE mask patch (49664,135168) is: 226.98
011A.tif Average pixel value for ANNOTATED AREA mask patch (49664,135168) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\49664_135168
011A.tif Average pixel value for TISUE mask patch (50176,135168) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (50176,135168) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\50176_135168
011A.tif Average pixel value for TISUE mask patch (50688,135168) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (50688,135168) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\50688_135168
011A.tif Average pixel value for TISUE mask patch (51200,135168) is: 255.00
011A.tif Average pixe

011A.tif Average pixel value for TISUE mask patch (49152,135680) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (49152,135680) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\49152_135680
011A.tif Average pixel value for TISUE mask patch (49664,135680) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (49664,135680) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\49664_135680
011A.tif Average pixel value for TISUE mask patch (50176,135680) is: 255.00
011A.tif Average pixel value for ANNOTATED AREA mask patch (50176,135680) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/cancerous_patches\50176_135680
011A.tif Average pixel value for TISUE mask patch (50688,135680) is: 255.00
011A.tif Average pixe

KeyboardInterrupt: 